In [1]:
import pandas as pd
import networkx as nx
import pandas as pd
from IPython.display import IFrame
import folium

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# Finding and Displaying train/car routes that are as fast as or faster than plane connections.

### Loading data

In [2]:
# define your path
my_path = "DOPP_GROUP25/data/"

# used to connect iata codes with cities and for coordinates
all_data = pd.read_csv(my_path + "clean/all_data.csv")
all_data.head()

,Airport,Country,icao_code,city,iata_code,airport_lat,airport_lng,iso_country_code,chronotrain_id,train_station_lat,train_station_lng,city_lat,city_lng,place_id,airport_duration,station_duration
0,Tirana Airport,Albania,LATI,Tirana,TIA,41.42190,19.71296,ALB,8302063.0,45.111682,8.670419,41.327546,19.818698,ChIJ28X6cAQxUBMRIDdlEK-SAAQ,26.933333,1090.766667
1,Graz Airport,Austria,LOWG,Graz,GRZ,46.99107,15.43963,AUT,2778067.0,47.066670,15.450000,47.070714,15.439504,ChIJu2UwF4c1bkcRm93f0tGKjv4,19.600000,5.583333
2,Innsbruck Airport,Austria,LOWI,Innsbruck,INN,47.26022,11.34396,AUT,2775220.0,47.262660,11.394540,47.269212,11.404102,ChIJc8r44c9unUcRDZsdKH0cIJ0,13.400000,6.216667
3,Klagenfurt Airport,Austria,LOWK,Klagenfurt,KLU,46.64500,14.32667,AUT,2774326.0,46.624720,14.305280,46.636460,14.312225,ChIJZX6PMEVzcEcRK41hjN-2fmg,5.616667,4.800000
4,Linz Airport,Austria,LOWL,Linz,LNZ,48.23322,14.18751,AUT,2772400.0,48.306390,14.286110,48.306940,14.285830,ChIJTYWZ-pWVc0cRxHV5VywpU3w,24.616667,0.316667


In [3]:
# car travel data
car_network = nx.read_gml(my_path + "clean/all_routes.gml")
car_network["Vienna"]["Innsbruck"]

{'duration': 308, 'distance': 477, 'type': 'car'}

In [4]:
# air travel data
airport_network = nx.read_gml(my_path + "airport_network.gml")
airport_network["VIE"]["INN"]

{'distance': 401733, 'duration': 115}

In [5]:
# train travel data
train_network = nx.read_gml(my_path + "city_train_network_duration.gml")
train_network["Vienna"]["Innsbruck"]

{'weight': 268}

In [6]:
# Renaming weight into duration for consistency with other types of transport
for u, v, data in train_network.edges(data=True):
    if 'weight' in data:
        data['duration'] = data.pop('weight')

In [7]:
train_network["Vienna"]["Innsbruck"]

{'duration': 268}

In [8]:
iata_codes = all_data['iata_code']
print("Unique IATA codes:\n", iata_codes.unique())
print("Number of unique IATA codes:", len(iata_codes))

Unique IATA codes:
 ['TIA' 'GRZ' 'INN' 'KLU' 'LNZ' 'SZG' 'VIE' 'GYD' 'MSQ' 'ANR' 'BRU' 'CRL'
 'LGG' 'OST' 'BOJ' 'SOF' 'VAR' 'PUY' 'RJK' 'SPU' 'ZAG' 'BRQ' 'PRG' 'AAL'
 'AAR' 'CPH' 'HEL' 'OUL' 'RVN' 'TMP' 'TKU' 'VAA' 'BIA' 'EGC' 'BIQ' 'BOD'
 'BES' 'LIL' 'LYS' 'MRS' 'MPL' 'NTE' 'NCE' 'BVA' 'CDG' 'ORY' 'RNS' 'SXB'
 'TLS' 'FMM' 'BER' 'BRE' 'CGN' 'DTM' 'DRS' 'DUS' 'FRA' 'FDH' 'HAM' 'HAJ'
 'FKB' 'LEJ' 'MUC' 'NUE' 'PAD' 'STR' 'NRN' 'ATH' 'SMI' 'SKG' 'BUD' 'DEB'
 'ORK' 'DUB' 'AOI' 'BRI' 'BGY' 'BLQ' 'BDS' 'CTA' 'CIY' 'FLR' 'GOA' 'SUF'
 'LIN' 'MXP' 'NAP' 'OLB' 'PMO' 'PEG' 'PSR' 'PSA' 'CIA' 'FCO' 'TPS' 'TSF'
 'TRS' 'TRN' 'VRN' 'RIX' 'KUN' 'VNO' 'LUX' 'KIV' 'TGD' 'AMS' 'EIN' 'GRQ'
 'MST' 'RTM' 'SKP' 'BGO' 'BOO' 'KRS' 'OSL' 'TRF' 'SVG' 'TRD' 'GDN' 'KTW'
 'KRK' 'POZ' 'SZZ' 'WAW' 'WMI' 'WRO' 'FAO' 'LIS' 'OPO' 'OTP' 'CLJ' 'IAS'
 'SBZ' 'TSR' 'BEG' 'INI' 'BTS' 'KSC' 'LJU' 'ALC' 'LEI' 'OVD' 'BCN' 'BIO'
 'GRO' 'XRY' 'SPC' 'MAD' 'AGP' 'REU' 'SCQ' 'SVQ' 'VLC' 'ZAZ' 'GOT' 'MMX'
 'ARN' 'BMA' 'NYO' 'BSL' 'GVA' 

### Looking for the shortest travel durations by train/car/plane

In [9]:
shortest_duration_by_train = nx.Graph() # to store shortest travel durations by train
shortest_duration_by_car = nx.Graph() # to store shortest travel durations by car
shortest_duration_by_plane = nx.Graph() # to store shortest travel durations by plane

for source_iata in iata_codes: # go through all source IATA codes
    for target_iata in iata_codes: # go through all target IATA codes
        if source_iata != target_iata: # source and target have to be different locations  
            
            source_city = all_data.loc[all_data['iata_code'] == source_iata, 'city'].values[0] # get the city corresponding to the source IATA 
            target_city = all_data.loc[all_data['iata_code'] == target_iata, 'city'].values[0] # get the city corresponding to the target IATA 
            if source_city != target_city: # source and target cities have to be different
                
                # checking if the edge exists in the train network
                if len(train_network[source_city]) > 0 and len(train_network[target_city]) > 0:
                    try:
                        path = nx.shortest_path(train_network, source_city, target_city, weight="duration") # find the shortest path by train
                        train_duration = nx.path_weight(train_network, path, weight="duration") # calculate the duration of the train ride
                    except nx.NetworkXNoPath:
                        train_duration = float('inf') # if there is no path, set a high value for duration
                else:
                    train_duration = float('inf')    

                # checking if the edge exists in the car network
                if car_network.has_edge(source_city, target_city):
                    car_duration = car_network[source_city][target_city]['duration'] ## get the duration of the car ride
                else:
                    car_duration = float('inf')  # set a high value if there is no car connection 
                    
                plane_duration = airport_network[source_iata][target_iata]['duration'] # get the duration of the flight

                # find the shortest duration among all types of transportation
                shortest_duration = min(train_duration, car_duration, plane_duration)

                if shortest_duration == train_duration:
                    shortest_duration_by_train.add_edge(source_city, target_city, duration=train_duration) # add edge to train graph
                elif shortest_duration == car_duration:
                    shortest_duration_by_car.add_edge(source_city, target_city, duration=car_duration) # add edge to car graph
                else:
                    shortest_duration_by_plane.add_edge(source_iata, target_iata, duration=plane_duration) # add edge to plane graph

## Display the resulting graphs

### Shorter (or equal) journey times by train (than by car or plane)

In [10]:
print("Train connections with the shortest duration\n:", shortest_duration_by_train.edges(data=True))

Train connections with the shortest duration
: [('Linz', 'Salzburg', {'duration': 68}), ('Linz', 'Vienna', {'duration': 75}), ('Salzburg', 'Munich', {'duration': 83}), ('Vienna', 'Brno', {'duration': 86}), ('Vienna', 'Bratislava', {'duration': 46}), ('Munich', 'Memmingen', {'duration': 63}), ('Munich', 'Nuremberg', {'duration': 62}), ('Minsk', 'Warsaw', {'duration': 25}), ('Antwerp', 'Brussels', {'duration': 42}), ('Antwerp', 'Liege', {'duration': 86}), ('Antwerp', 'Bari', {'duration': 124}), ('Antwerp', 'Amsterdam', {'duration': 80}), ('Antwerp', 'Rotterdam', {'duration': 41}), ('Brussels', 'Charleroi', {'duration': 50}), ('Brussels', 'Liege', {'duration': 44}), ('Brussels', 'Ostend', {'duration': 69}), ('Brussels', 'Paris', {'duration': 82}), ('Brussels', 'Bari', {'duration': 82}), ('Brussels', 'Maastricht', {'duration': 78}), ('Brussels', 'Rotterdam', {'duration': 70}), ('Liege', 'Cologne', {'duration': 64}), ('Liege', 'Düsseldorf', {'duration': 90}), ('Liege', 'Bari', {'duration': 

In [11]:
train_network['Linz']['Salzburg']

{'duration': 68}

In [12]:
car_network['Linz']['Salzburg']

{'duration': 85, 'distance': 132, 'type': 'car'}

In [13]:
airport_network['LNZ']['SZG']

{'distance': 100690, 'duration': 92}

In [14]:
map_center = [50.0, 15.0]
mymap_train = folium.Map(location=map_center, zoom_start=4.1)

# iterating over the edges of the graph and add them to the map
for edge in shortest_duration_by_train.edges(data=True):
    source_city, target_city, data = edge
    duration = data.get("duration", "N/A")

    # adding a line connecting the source and target cities using train station coordinates
    source_lat = all_data.loc[all_data['city'] == source_city, 'train_station_lat'].values[0]
    source_lng = all_data.loc[all_data['city'] == source_city, 'train_station_lng'].values[0]

    target_lat = all_data.loc[all_data['city'] == target_city, 'train_station_lat'].values[0]
    target_lng = all_data.loc[all_data['city'] == target_city, 'train_station_lng'].values[0]

    folium.PolyLine(
        locations=[(source_lat, source_lng), (target_lat, target_lng)],
        color='green',
        popup=f"Duration: {duration}",
        weight=2.5
    ).add_to(mymap_train)

    
# saving the map as an HTML file
map_filename_train = "shortest_duration_by_train_map.html"
mymap_train.save(map_filename_train)

# display the map
IFrame(src=map_filename_train, width=800, height=600)

### Shorter (or equal) journey times by car (than by train or plane)

In [15]:
print("Car connections with the shortest duration\n:", shortest_duration_by_car.edges(data=True))

Car connections with the shortest duration
: [('Klagenfurt', 'Ljubljana', {'duration': 88}), ('Ljubljana', 'Trieste', {'duration': 76}), ('Antwerp', 'Charleroi', {'duration': 81}), ('Antwerp', 'Ostend', {'duration': 83}), ('Antwerp', 'Lille', {'duration': 93}), ('Antwerp', 'Eindhoven', {'duration': 66}), ('Antwerp', 'Maastricht', {'duration': 89}), ('Charleroi', 'Liege', {'duration': 60}), ('Charleroi', 'Lille', {'duration': 79}), ('Charleroi', 'Maastricht', {'duration': 81}), ('Ostend', 'Lille', {'duration': 66}), ('Eindhoven', 'Liege', {'duration': 84}), ('Eindhoven', 'Düsseldorf', {'duration': 83}), ('Eindhoven', 'Weeze', {'duration': 73}), ('Maastricht', 'Liege', {'duration': 32}), ('Maastricht', 'Cologne', {'duration': 81}), ('Maastricht', 'Düsseldorf', {'duration': 78}), ('Pula', 'Rijeka', {'duration': 88}), ('Rijeka', 'Trieste', {'duration': 84}), ('Brno', 'Bratislava', {'duration': 92}), ('Aalborg', 'Aarhus', {'duration': 82}), ('Memmingen', 'Friedrichshafen', {'duration': 65})

In [16]:
car_network['Klagenfurt']['Ljubljana']

{'duration': 88, 'distance': 86, 'type': 'car'}

In [17]:
train_network['Klagenfurt']['Ljubljana']

{'duration': 146}

In [18]:
airport_network['KLU']['LJU']

{'distance': 47910, 'duration': 88}

In [19]:
map_center = [50.0, 15.0]
mymap_car = folium.Map(location=map_center, zoom_start=4.1)

# iterating over the edges of the graph and add them to the map
for edge in shortest_duration_by_car.edges(data=True):
    source_city, target_city, data = edge
    duration = data.get("duration", "N/A")

    # adding a line connecting the source and target cities using city coordinates
    source_lat = all_data.loc[all_data['city'] == source_city, 'city_lat'].values[0]
    source_lng = all_data.loc[all_data['city'] == source_city, 'city_lng'].values[0]

    target_lat = all_data.loc[all_data['city'] == target_city, 'city_lat'].values[0]
    target_lng = all_data.loc[all_data['city'] == target_city, 'city_lng'].values[0]

    folium.PolyLine(
        locations=[(source_lat, source_lng), (target_lat, target_lng)],
        color='red', 
        popup=f"Duration: {duration}",
        weight=2.5
    ).add_to(mymap_car)

# saving the map as an HTML file
map_filename_car = "shortest_duration_by_car_map.html"
mymap_car.save(map_filename_car)

# display the map
IFrame(src=map_filename_car, width=800, height=600)

### Shorter (or equal) journey times by plane (than by train or car)

In [20]:
print("Air connections with the shortest duration\n:", shortest_duration_by_plane.edges(data=True))

Air connections with the shortest duration
: [('TIA', 'GRZ', {'duration': 139}), ('TIA', 'INN', {'duration': 156}), ('TIA', 'KLU', {'duration': 140}), ('TIA', 'LNZ', {'duration': 152}), ('TIA', 'SZG', {'duration': 153}), ('TIA', 'VIE', {'duration': 145}), ('TIA', 'GYD', {'duration': 280}), ('TIA', 'MSQ', {'duration': 201}), ('TIA', 'ANR', {'duration': 207}), ('TIA', 'BRU', {'duration': 206}), ('TIA', 'CRL', {'duration': 203}), ('TIA', 'LGG', {'duration': 200}), ('TIA', 'OST', {'duration': 214}), ('TIA', 'BOJ', {'duration': 135}), ('TIA', 'SOF', {'duration': 110}), ('TIA', 'VAR', {'duration': 138}), ('TIA', 'PUY', {'duration': 131}), ('TIA', 'RJK', {'duration': 130}), ('TIA', 'SPU', {'duration': 113}), ('TIA', 'ZAG', {'duration': 128}), ('TIA', 'BRQ', {'duration': 153}), ('TIA', 'PRG', {'duration': 166}), ('TIA', 'AAL', {'duration': 229}), ('TIA', 'AAR', {'duration': 222}), ('TIA', 'CPH', {'duration': 212}), ('TIA', 'HEL', {'duration': 249}), ('TIA', 'OUL', {'duration': 288}), ('TIA', '

In [21]:
airport_network['TIA']['GRZ']

{'distance': 706492, 'duration': 139}

In [22]:
car_network['Tirana']['Graz']

{'duration': 769, 'distance': 1041, 'type': 'car'}

In [23]:
train_network['Tirana'] #Tirana has no edges for train travel 

AtlasView({})

In [24]:
map_center = [50.0, 15.0]
mymap_plane = folium.Map(location=map_center, zoom_start=4)

# iterating over the edges of the graph and add them to the map
for edge in shortest_duration_by_plane.edges(data=True):
    source_iata, target_iata, data = edge
    duration = data.get("duration", "N/A")

    # adding a line connecting the source and target cities using airport coordinates
    source_lat = all_data.loc[all_data['iata_code'] == source_iata, 'airport_lat'].values[0]
    source_lng = all_data.loc[all_data['iata_code'] == source_iata, 'airport_lng'].values[0]

    target_lat = all_data.loc[all_data['iata_code'] == target_iata, 'airport_lat'].values[0]
    target_lng = all_data.loc[all_data['iata_code'] == target_iata, 'airport_lng'].values[0]

    folium.PolyLine(
        locations=[(source_lat, source_lng), (target_lat, target_lng)],
        color='blue',  
        popup=f"Duration: {duration}",
        weight=0.1
    ).add_to(mymap_plane)

# saving the map as an HTML file
map_filename_plane = "shortest_duration_by_plane_map.html"
mymap_plane.save(map_filename_plane)

# display the map
IFrame(src=map_filename_plane, width=800, height=600)
